# Computational Theory Assement Problems

This notebook implements the SHA-256 cryptographic hash function from scratch, following the Secure Hash Standard (FIPS 180-4). Each problem builds upon the previous one, culminating in a complete working implementation of SHA-256.

## Introduction to SHA-256

SHA-256 (Secure Hash Algorithm 256-bit) is a cryptographic hash function that takes an input of any length and produces a fixed 256-bit (32-byte) output called a hash or digest. It has several important properties:

- **Deterministic**: The same input always produces the same output
- **Fast to compute**: Efficient to calculate the hash for any given message
- **One-way function**: Computationally infeasible to reverse (find input from hash)
- **Collision resistant**: Extremely difficult to find two different inputs with the same hash
- **Avalanche effect**: Small change in input causes significant change in output

SHA-256 is widely used in security applications including SSL/TLS, cryptocurrencies (Bitcoin), digital signatures, and password storage.


## Problem 1: Binary Words and Operations

This problem implements fundamental bitwise operations used in the SHA-256 cryptographic hash function. These operations manipulate 32-bit words at the bit level to provide the mathematical foundation for secure hashing.

### Background on Bitwise Operations

The Secure Hash Standard (SHS) defines several bitwise functions that operate on 32-bit unsigned integers. These functions are essential building blocks for the SHA-256 algorithm:

#### 1. **Parity (x ⊕ y ⊕ z)**
The XOR (exclusive or) operation returns 1 when an odd number of input bits are 1, and 0 when an even number are 1. This creates a balanced, non-linear transformation of the input bits.

**Truth table for XOR:**
| x | y | z | x⊕y⊕z |
|---|---|---|-------|
| 0 | 0 | 0 |   0   |
| 0 | 0 | 1 |   1   |
| 0 | 1 | 0 |   1   |
| 0 | 1 | 1 |   0   |
| 1 | 0 | 0 |   1   |
| 1 | 0 | 1 |   0   |
| 1 | 1 | 0 |   0   |
| 1 | 1 | 1 |   1   |

#### 2. **Ch (Choose): (x ∧ y) ⊕ (¬x ∧ z)**
This function selects bits from y or z based on the corresponding bit in x. If x bit is 1, take from y; if 0, take from z. This creates a "choice" operation that adds non-linearity to the hash function.

**Example:** If x = 1100, y = 1010, z = 0110
- Positions where x=1 (first two bits): take from y → 10
- Positions where x=0 (last two bits): take from z → 10
- Result: 1010

#### 3. **Maj (Majority): (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)**
Returns the majority bit value at each position. If two or more inputs have a 1 at a position, the result is 1; otherwise 0. This operation provides a form of "voting" among the three inputs.

#### 4. **ROTR (Rotate Right)**
Circular right shift where bits that fall off the right end wrap around to the left. This preserves all bits while changing their positions, ensuring no information is lost.

**Example:** ROTR³(11000000...000) = 00011000...000

#### 5. **SHR (Shift Right)**
Logical right shift where bits that fall off are discarded and zeros fill in from the left. This operation loses information.

#### 6. **Sigma Functions (Σ₀, Σ₁, σ₀, σ₁)**
These functions combine multiple rotations and shifts using XOR. They create complex, non-linear transformations that are essential for SHA-256's security:
- **Σ₀ and Σ₁** (uppercase): Used in the compression function
- **σ₀ and σ₁** (lowercase): Used in the message schedule

### Why These Operations Matter

These bitwise operations provide several crucial properties:
1. **Non-linearity**: Makes it impossible to predict output from input using linear algebra
2. **Diffusion**: Spreads the influence of each input bit across many output bits
3. **Confusion**: Obscures the relationship between input and output
4. **Efficiency**: All operations are fast on modern processors

### Implementation

The following implementations use NumPy's `uint32` type to ensure all operations work with 32-bit unsigned integers, as required by the SHA-256 specification.

In [124]:
import numpy as np
np.seterr(over='ignore')

{'divide': 'warn', 'over': 'ignore', 'under': 'ignore', 'invalid': 'warn'}

In [125]:
#Global SHA-256 constants
sha256_constants = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
    0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
    0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
    0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
    0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
    0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
    0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
    0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
    0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
    ]
#global initial hash values for SHA-256 (current values in problem 4)
sha256_initial_hash_values = [
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
    ]

In [126]:
#1.
# ^ = bitwise XOR
def Parity(x,y,z):
    """
    Performs bitwise XOR(exclusive or) operation on each bit position of the 3 values
    x == 0 y == 0 z == 0 result == 0
    x == 1 y == 0 z == 0 result == 1 (returns 1 when odd number of 1's)
    x == 1 y == 1 z == 0 result == 0 (returns 0 when even number of 1's)
    x == 1 y == 1 z == 1 result == 1
    Arg:

    Parameters: 
    X: int 
    First 32 bit unsigned integer 
    Y: int
    Second 32 bit unsigned integer
    Z: int 
    Third 32 bit unsigned  integer

    Returns:
    np.uint32
        value made up of results for each bit position of the 3 input XOR
    """
    return np.uint32(x) ^ np.uint32(y) ^ np.uint32(z)

#Test 
#input
x = 0b1101  # 13 
y = 0b0111  # 7 
z = 0b0000  # 0 
#expected 
expected = 0b1010 # 10
#actual
actual = Parity(x,y,z)
print(f"Input: x={x} ({bin(x)}), y={y} ({bin(y)}), z={z} ({bin(z)})")
print(f"Result: {actual} ({bin(actual)})")
print(f"Expected: {expected} ({bin(expected)})")
print(f"Test Passed: {actual == expected}")


Input: x=13 (0b1101), y=7 (0b111), z=0 (0b0)
Result: 10 (0b1010)
Expected: 10 (0b1010)
Test Passed: True


In [127]:
#2.
# & = bitwise and 
# ~ = not operator (flips all bits i.e 0x00000000 -> 0xFFFFFFFF)
def Ch(x,y,z):
    """
    returns new value built from the taken bits of y and z based on the state of x

    For each bit position in x check if the bit is 1.
    if it is, take the corresponding bit in y 
    if not, take the corresponding bit in z

    Parameters
    ----------
    X: int 
    First 32 bit unsigned integer 
    Y: int
    Second 32 bit unsigned integer
    Z: int 
    Third 32 bit unsigned integer

    Returns
    ---------
    np.unit32 
        value made from z and y based on the state of x
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return (x & y) ^ (~x & z)

# Test 
x = 0b1100  # 12 
y = 0b1010  # 10 
z = 0b0110  # 6 

#expected result
expected = 0b1010  # 10
#actual result
actual = Ch(x, y, z)
print(f"Input: x={x} ({bin(x)}), y={y} ({bin(y)}), z={z} ({bin(z)})")
print(f"Result: {actual} ({bin(actual)})")
print(f"Expected: {expected} ({bin(expected)})")
print(f"Test Passed: {actual == expected}")


Input: x=12 (0b1100), y=10 (0b1010), z=6 (0b110)
Result: 10 (0b1010)
Expected: 10 (0b1010)
Test Passed: True


In [128]:
#3.
def Maj(x,y,z):
    """ 
    Returns 32 bit integer bitwise majority

    For each bit position, returns 1 if two or more of the three inputs 
    have that bit set to 1 else returns 0.

    Parameters
    ---------
    X: int 
    First 32 bit unsigned integer 
    Y: int
    Second 32 bit unsigned integer
    Z: int 
    Third 32 bit unsigned integer

    Returns
    ------- 
    np.uint32
        value made up of the majority of each bit position of the 3 values
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    #For each bit position perform a bitwise AND between each of the 3 values
    #which returns 1 if both bits are 1 or 0 in all other cases, then perform a 3 input XOR
    #using the values returned from the AND comparisons 

    return (x & y) ^ (x & z) ^ (y & z)

# Test 
x = 0b1100  # 12
y = 0b1010  # 10
z = 0b1001  # 9

expected = 0b1000  # 8
actual = Maj(x, y, z)
print(f"Input: x={x} ({bin(x)}), y={y} ({bin(y)}), z={z} ({bin(z)})")
print(f"Result: {actual} ({bin(actual)})")
print(f"Expected: {expected} ({bin(expected)})")
print(f"Test Passed: {actual == expected}")

Input: x=12 (0b1100), y=10 (0b1010), z=9 (0b1001)
Result: 8 (0b1000)
Expected: 8 (0b1000)
Test Passed: True


In [129]:
#4.
# | = bitwise inclusive or
# W = number of bits 
# >> = shifts each bit to the right bits on the right fall off and are replaced on the left by 0's
# i.e n = 3   11000-->00011
# << = shifts each bit to the left bits on the left fall off and are replaced on the right by 0's
#i.e n = 3 11001->01000
def ROTR(x,n):
    """ 
    Executes  rotate right (circular right shift) operation on 32 bit unsigned integer
    Rotates all bits in X to the right by n.Any bits that fall off are wrapped to the left
    end

    Parameters
    ---------
    X: int 
    32 bit unsigned integer 
    N: int 
    number of bits to rotate x by 0-31

    Returns
    ------- 
    np.uint32
        Resulting rotation of x by n bit positions
        
    """
    x = np.uint32(x)
    n = np.uint32(n)
    w = 32
    #shifts each bit in x to the right by n positions
    #left shift the bits of x by the bit size of x-n(32-n) 
    #combine both shifted values using bitwise OR to return the rotated result
    #& 0xFFFFFFFF Ensures results remain within 32bits by zeroing any bits over 32
    # https://stackoverflow.com/questions/10493411/what-is-bit-masking  I used this to understand how bitmasking works and why it might be nessarcary
    return ((x >> n) | (x << (w - n))) & 0xFFFFFFFF 

#test 
#inputs
x = 0b11000000000000000000000000000000  
n = 3 # rotate by 3
#outputs
expected = 0b00011000000000000000000000000000  # Rotated right by 3
actual = ROTR(x, n)
print(f"Input: x={x} ({bin(x)}), n={n}")
print(f"Result: {actual} (0b{actual:032b})")
print(f"Expected: {expected} (0b{expected:032b})")
print(f"Test Passed: {actual == expected}")

Input: x=3221225472 (0b11000000000000000000000000000000), n=3
Result: 402653184 (0b00011000000000000000000000000000)
Expected: 402653184 (0b00011000000000000000000000000000)
Test Passed: True


In [130]:
def SHR(x,n):
    """Executes the right shift operation on a 32-bit integer.
    shifts each bit position of x by n positions.Any bit that falls off is replaced by a 0 from the left end
    
    Parameters
    ---------
    X: int 
    32 bit unsigned integer 
    N: int 
    number of bits to shift x by 0-31

    Returns
    ------- 
    np.uint32
        Resulting shift of x by n bit positions
    
    """
    x = np.uint32(x)
    n = np.uint32(n)
    #shift bit positions of x by n positions and return the shifted 32 bit integer
    return np.uint32( x >> n)

# Test
# inputs
x = 0b11000000000000000000000000000000  
n = 3# shift by 3
# outputs
expected = 0b00011000000000000000000000000000  # Shifted right by 3
actual = SHR(x, n)


print(f"Input: x={x} (0b{x:032b}), n={n}")
print(f"Result: {actual} (0b{actual:032b})")
print(f"Expected: {expected} (0b{expected:032b})")
print(f"Test Passed: {actual == expected}")

Input: x=3221225472 (0b11000000000000000000000000000000), n=3
Result: 402653184 (0b00011000000000000000000000000000)
Expected: 402653184 (0b00011000000000000000000000000000)
Test Passed: True


In [131]:
def Sigma0(x):
    """
    SHA-256 Sigma0 (Σ₀)

    Performs 3 input XOR using 3 seperate right rotated values of x,
      each with differing rotation sizes(2, 13, and 22 bit positions)

    Parameters
    ---------
    X: int 
    32 bit unsigned integer 

    Returns
    -------
    np.uint32
        Result of the 3 input XOR on the rotated values
    """
    x = np.uint32(x)
    return ROTR(x,2) ^ ROTR(x,13) ^ ROTR(x,22)

#test 
#input
x = 0b10101010101010101010101010101010  
#outputs
expected = 0b01010101010101010101010101010101
actual = Sigma0(x)

print(f"Input: x={x} (0b{x:032b})")
print(f"Result: {actual} (0b{actual:032b})")
print(f"Expected: {expected} (0b{expected:032b})")
print(f"Test Passed: {actual == expected}")

Input: x=2863311530 (0b10101010101010101010101010101010)
Result: 1431655765 (0b01010101010101010101010101010101)
Expected: 1431655765 (0b01010101010101010101010101010101)
Test Passed: True


In [132]:
#5.
def Sigma1(x):
    """
    SHA-256 Sigma1 (Σ₁) function

    Performs 3 input XOR using 3 seperate right rotated values of x,
      each with differing rotation sizes(6, 11, and 25 bit positions) 
      

    Parameters
    ---------
    X: int 
    32 bit unsigned integer 

    Returns
    --------
    np.uint32
        Result of the 3 input XOR on the 3 rotated values 
    """
    x = np.uint32(x)
    return ROTR(x,6) ^ ROTR(x,11) ^ ROTR(x,25)

#test
#input
x = 0b10000000000000000000000000000000  
#outputs
expected = 0b00000010000100000000000001000000 
actual = Sigma1(x)

print(f"Input: x=0x{x:08X} (0b{x:032b})")
print(f"Result: 0x{actual:08X} (0b{actual:032b})")
print(f"Expected: 0x{expected:08X} (0b{expected:032b})")
print(f"Test Passed: {actual == expected}")

Input: x=0x80000000 (0b10000000000000000000000000000000)
Result: 0x02100040 (0b00000010000100000000000001000000)
Expected: 0x02100040 (0b00000010000100000000000001000000)
Test Passed: True


In [133]:
#6.
def Sigma0_2(x):
    """
    SHA-256 sigma0 (σ₀)

    Performs 3 input XOR using 2 seperate right rotated values of x,
      each with differing rotation sizes(7, and 18 bit positions) 
      and one right shifted value of x shifted by 3 bit positions

    Parameters
    ---------
    X: int 
    32 bit integer 

    Returns
    --------
    np.uint32
        Result of the 3 input XOR on the 2 rotated values of x and 1 right shifted value of x
    """
    x = np.uint32(x)
    return ROTR(x,7) ^ ROTR(x,18) ^ SHR(x,3)
#test
#input
x = 0b10000000000000000000000000000000 
#outputs
expected = 0b00010001000000000010000000000000 
actual = Sigma0_2(x)


print(f"Input: x=0x{x:08X} (0b{x:032b})")
print(f"Result: 0x{actual:08X} (0b{actual:032b})")
print(f"Expected: 0x{expected:08X} (0b{expected:032b})")
print(f"Test Passed: {actual == expected}")

Input: x=0x80000000 (0b10000000000000000000000000000000)
Result: 0x11002000 (0b00010001000000000010000000000000)
Expected: 0x11002000 (0b00010001000000000010000000000000)
Test Passed: True


In [134]:
#7.
def Sigma1_2(x):
    """
    SHA-256 sigma1 (σ₁)
    Performs 3 input XOR using 2 seperate right rotated values of x
      each with differing rotation sizes(17, and 19 bit positions) 
      and one right shifted value of x shifted by 10 bit positions
      
    Parameters
    ---------
    X: int 
    32 bit integer 

    Returns
    --------
    np.uint32
        Result of the 3 input XOR on the 2 rotated values of x and 1 right shifted value of x
    """
    x = np.uint32(x)
    return ROTR(x,17) ^ ROTR(x,19) ^ SHR(x,10)
#test
#input
x = 0b10000000000000000000000000000000  
#outputs
expected = 0b00000000001000000101000000000000 
actual = Sigma1_2(x)

print(f"Input: x= {x} (0b{x:032b})")
print(f"Result: {actual} (0b{actual:032b})")
print(f"Expected: {expected} (0b{expected:032b})")
print(f"Test Passed: {actual == expected}")

Input: x= 2147483648 (0b10000000000000000000000000000000)
Result: 2117632 (0b00000000001000000101000000000000)
Expected: 2117632 (0b00000000001000000101000000000000)
Test Passed: True


### Testing Problem 1 Functions

The following tests verify that our bitwise operations work correctly with various inputs including edge cases, standard operations, and patterns. Each function is tested with multiple scenarios to ensure correctness.


In [135]:
def test_problem1_functions():
    """
    Comprehensive test suite for Problem 1 bitwise operations.
    
    Tests each function with multiple cases including edge cases,
    standard operations, and values from the SHA-256 specification.
    """
  
    print("Testing Problem 1: Binary Words and Operations")
    
    # Test 1: Parity
    print("\n--- Testing Parity(x, y, z) ---")
    test_cases_parity = [
        (0b1101, 0b0111, 0b0000, 0b1010),  # Mixed bits
        (0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF),  # All 1s
        (0, 0, 0, 0),  # All 0s
        (0xAAAAAAAA, 0x55555555, 0, 0xFFFFFFFF),  # Alternating patterns
    ]
    
    for x, y, z, expected in test_cases_parity:
        result = Parity(x, y, z)
        status = "PASS" if result == expected else "FAIL"
        print(f"{status}: Parity(0x{x:08X}, 0x{y:08X}, 0x{z:08X}) = 0x{result:08X} (expected 0x{expected:08X})")
    
    # Test 2: Ch (Choose)
    print("\n--- Testing Ch(x, y, z) ---")
    test_cases_ch = [
        (0b1100, 0b1010, 0b0110, 0b1010),  # Basic choose
        (0xFFFFFFFF, 0x12345678, 0x87654321, 0x12345678),  # All x bits = 1
        (0, 0x12345678, 0x87654321, 0x87654321),  # All x bits = 0
        (0xF0F0F0F0, 0xAAAAAAAA, 0x55555555, 0xA5A5A5A5),  # Mixed pattern
    ]
    
    for x, y, z, expected in test_cases_ch:
        result = Ch(x, y, z)
        status = "PASS" if result == expected else "FAIL"
        print(f"{status}: Ch(0x{x:08X}, 0x{y:08X}, 0x{z:08X}) = 0x{result:08X} (expected 0x{expected:08X})")
    
    # Test 3: Maj (Majority)
    print("\n--- Testing Maj(x, y, z) ---")
    test_cases_maj = [
        (0b1100, 0b1010, 0b1001, 0b1000),  # Basic majority
        (0xFFFFFFFF, 0xFFFFFFFF, 0, 0xFFFFFFFF),  # Two all 1s
        (0, 0, 0xFFFFFFFF, 0),  # Two all 0s
        (0xAAAAAAAA, 0xCCCCCCCC, 0xF0F0F0F0, 0xE8E8E8E8),  # Complex pattern
    ]
    
    for x, y, z, expected in test_cases_maj:
        result = Maj(x, y, z)
        status = "PASS" if result == expected else "FAIL"
        print(f"{status}: Maj(0x{x:08X}, 0x{y:08X}, 0x{z:08X}) = 0x{result:08X} (expected 0x{expected:08X})")
    
    # Test 4: ROTR (Rotate Right)
    print("\n--- Testing ROTR(x, n) ---")
    test_cases_rotr = [
        (0xC0000000, 3, 0x18000000),  # Basic rotation
        (0x12345678, 8, 0x78123456),  # Byte-aligned rotation
        (0xFFFFFFFF, 16, 0xFFFFFFFF),  # All 1s
        (0x80000001, 1, 0xC0000000),  # Wrap around
    ]
    
    for x, n, expected in test_cases_rotr:
        result = ROTR(x, n)
        status = "PASS" if result == expected else "FAIL"
        print(f"{status}: ROTR(0x{x:08X}, {n}) = 0x{result:08X} (expected 0x{expected:08X})")
    
    # Test 5: SHR (Shift Right)
    print("\n--- Testing SHR(x, n) ---")
    test_cases_shr = [
        (0xC0000000, 3, 0x18000000),  # Basic shift
        (0xFFFFFFFF, 8, 0x00FFFFFF),  # Zeros fill from left
        (0x12345678, 16, 0x00001234),  # Shift half word
        (0xAAAAAAAA, 1, 0x55555555),  # Pattern shift
    ]
    
    for x, n, expected in test_cases_shr:
        result = SHR(x, n)
        status = "PASS" if result == expected else "FAIL"
        print(f"{status}: SHR(0x{x:08X}, {n}) = 0x{result:08X} (expected 0x{expected:08X})")
    
    # Test 6-9: Sigma functions
    print("\n--- Testing Sigma Functions ---")
    test_value = 0x12345678
    
    print(f"Σ₀(0x{test_value:08X}) = 0x{Sigma0(test_value):08X}")
    print(f"Σ₁(0x{test_value:08X}) = 0x{Sigma1(test_value):08X}")
    print(f"σ₀(0x{test_value:08X}) = 0x{Sigma0_2(test_value):08X}")
    print(f"σ₁(0x{test_value:08X}) = 0x{Sigma1_2(test_value):08X}")
    
    # Verify with alternating bit pattern
    test_value2 = 0xAAAAAAAA
    print(f"\nΣ₀(0x{test_value2:08X}) = 0x{Sigma0(test_value2):08X}")
    print(f"Σ₁(0x{test_value2:08X}) = 0x{Sigma1(test_value2):08X}")
    print(f"σ₀(0x{test_value2:08X}) = 0x{Sigma0_2(test_value2):08X}")
    print(f"σ₁(0x{test_value2:08X}) = 0x{Sigma1_2(test_value2):08X}")
    

    print("Problem 1 Tests Complete")
   

# Run tests
test_problem1_functions()

Testing Problem 1: Binary Words and Operations

--- Testing Parity(x, y, z) ---
PASS: Parity(0x0000000D, 0x00000007, 0x00000000) = 0x0000000A (expected 0x0000000A)
PASS: Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF) = 0xFFFFFFFF (expected 0xFFFFFFFF)
PASS: Parity(0x00000000, 0x00000000, 0x00000000) = 0x00000000 (expected 0x00000000)
PASS: Parity(0xAAAAAAAA, 0x55555555, 0x00000000) = 0xFFFFFFFF (expected 0xFFFFFFFF)

--- Testing Ch(x, y, z) ---
PASS: Ch(0x0000000C, 0x0000000A, 0x00000006) = 0x0000000A (expected 0x0000000A)
PASS: Ch(0xFFFFFFFF, 0x12345678, 0x87654321) = 0x12345678 (expected 0x12345678)
PASS: Ch(0x00000000, 0x12345678, 0x87654321) = 0x87654321 (expected 0x87654321)
PASS: Ch(0xF0F0F0F0, 0xAAAAAAAA, 0x55555555) = 0xA5A5A5A5 (expected 0xA5A5A5A5)

--- Testing Maj(x, y, z) ---
PASS: Maj(0x0000000C, 0x0000000A, 0x00000009) = 0x00000008 (expected 0x00000008)
PASS: Maj(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000) = 0xFFFFFFFF (expected 0xFFFFFFFF)
PASS: Maj(0x00000000, 0x00000000, 0xFFFFFFF

## Problem 2: Fractional Parts of Cube Roots

The SHA-256 algorithm uses 64 constant values in its compression function. To ensure these constants have no hidden weaknesses or backdoors, they are derived from a natural mathematical source: the cube roots of prime numbers.
### Why Prime Numbers and Cube Roots?

**Prime numbers** are fundamental in mathematics and have no special patterns that could be exploited. By using the first 64 primes (2, 3, 5, 7, 11, ...), we ensure a systematic, verifiable method that anyone can reproduce.

**Cube roots** (∛n) provide irrational numbers with no repeating patterns in their decimal/binary representations. The fractional parts of these cube roots appear random but are completely deterministic.

### Mathematical Process

For each prime number p:

1. **Calculate cube root**: ∛p
   - Example: ∛2 = 1.259921049894873...

2. **Extract fractional part**: ∛p - ⌊∛p⌋
   - Example: 1.259921049894873... - 1 = 0.259921049894873...

3. **Convert to 32-bit representation**: fractional_part × 2³²
   - Example: 0.259921049894873... × 4,294,967,296 = 1,116,352,408.18...

4. **Take integer part and convert to hex**
   - Example: 1,116,352,408 = 0x428a2f98

This gives us the first 32 bits of the fractional part as a hexadecimal constant that appears random but is mathematically derived.

### Prime Number Generation

To generate the first n prime numbers, we use **trial division**, a straightforward algorithm:

1. Start with candidate number 2
2. For each candidate, check if it's divisible by any number from 2 to √candidate
3. If no divisors are found, the number is prime
4. Continue until n primes are found

**Optimization**: We only need to check up to √n because if n has a factor greater than √n, it must also have a factor less than √n (since a × b = n and if a > √n then b < √n).

### Why This Matters for Security

Using mathematically derived constants:
- **Prevents backdoors**: No way to choose constants that create hidden weaknesses
- **Verifiable**: Anyone can reproduce the constants
- **Appears random**: No patterns that could be exploited
- **Standardized**: International standard ensures everyone uses the same values



In [136]:
#Part 1
"""
Generates the first n prime numbers and returns them in an list

Parameters
----------
n: int
    number of prime numbers to generate

Returns
-------
list
    list of the first n prime numbers
    
References: https://www.geeksforgeeks.org/dsa/generate-and-print-first-n-prime-numbers/ broke down how to find the first N prime numbers
"""
def primes(n):
    if n <= 0:
        return []
    #current number of found primes   
    x = 0
    #current number to check for prime
    y = 2
    #array to store prime numbers in
    primes = []
    #prime flagger
    flag = False
    #loop until n primes have been found
    while(x < n):
        #assume the number is prime
        flag = True
        #loop over and check if y is divisible by any number from 2 to sqrt(y)
        #if y is not prime it can be factored into 2 factors a and b
        #y = a*b and at least one of those factors must be less than or equal to sqrt(y)
        for j in range(2, int(np.floor(np.sqrt(y))) + 1):
            #check if y is divisible by j
            if(y % j == 0):
                #if it is then y is not prime
                flag = False
                break
        #if the number is still flagged as prime as it passed all divisibility tests
        if(flag):
            #increment count of found primes 
            #and add the prime to the list
            x+=1
            primes.append(y)
        #increment number to check for prime
        y+=1    
    return primes




In [137]:
#Part 2-4
def generate_sha256_constants(n):
    """
    Generate first n prime numbers then Calculates the cube roots.
    Extracts the fractional part of a number by subtracting the floored value from the original number.
    Shift the fractional part by 32 binary places(2^32) and convert to integer to  remove any remaining decimal values.
    convert the resulting integers to hex and save as an array.
    Parameters
    ----------
    cubeRoots_primes64: float
        number to extract fractional part from

    Returns
    -------
    list
        fractional part of cubeRoots_primes64 shifted by 32 bit positions and converted to hex
    """
    # Step 1: Generate first n primes
    primes_list = primes(n)
    
    # Step 2: Calculate cube roots
    cube_roots = np.cbrt(primes_list)
    hex_roots = []
    #loop over each cube root of the first 64 primes
    for root in cube_roots:
        #extract fractional part by subtracting the value before the decimal from the original value
        fractional_part = root - np.floor(root)
        #shift the fractional part by 32 bit positions
        shifted_fractional = fractional_part * (2**32)
        #convert to integer to remove any remaining decimal values
        integer_fractional = int(shifted_fractional)
        #Part 4
        #convert to hex and append to array
        hex_roots.append(hex(integer_fractional))
    return hex_roots
hex_values = generate_sha256_constants(64)
print(hex_values)

['0x428a2f98', '0x71374491', '0xb5c0fbcf', '0xe9b5dba5', '0x3956c25b', '0x59f111f1', '0x923f82a4', '0xab1c5ed5', '0xd807aa98', '0x12835b01', '0x243185be', '0x550c7dc3', '0x72be5d74', '0x80deb1fe', '0x9bdc06a7', '0xc19bf174', '0xe49b69c1', '0xefbe4786', '0xfc19dc6', '0x240ca1cc', '0x2de92c6f', '0x4a7484aa', '0x5cb0a9dc', '0x76f988da', '0x983e5152', '0xa831c66d', '0xb00327c8', '0xbf597fc7', '0xc6e00bf3', '0xd5a79147', '0x6ca6351', '0x14292967', '0x27b70a85', '0x2e1b2138', '0x4d2c6dfc', '0x53380d13', '0x650a7354', '0x766a0abb', '0x81c2c92e', '0x92722c85', '0xa2bfe8a1', '0xa81a664b', '0xc24b8b70', '0xc76c51a3', '0xd192e819', '0xd6990624', '0xf40e3585', '0x106aa070', '0x19a4c116', '0x1e376c08', '0x2748774c', '0x34b0bcb5', '0x391c0cb3', '0x4ed8aa4a', '0x5b9cca4f', '0x682e6ff3', '0x748f82ee', '0x78a5636f', '0x84c87814', '0x8cc70208', '0x90befffa', '0xa4506ceb', '0xbef9a3f7', '0xc67178f2']


### Verifying Against Official Constants

The following test compares our generated constants against the official SHA-256 constants from FIPS 180-4. A perfect match confirms our implementation correctly derives the constants using the specified mathematical process.

In [138]:
#part 5
def  SHScomparison(generated_constants):
    """
    Compares the generated SHA-512 constants with the official SHA-512 constants
    
    Parameters
    ----------
    generated_constants : list
        The list of generated SHA-512 constants to compare
    Returns
    -------
    bool
        True if the generated constants match the official constants, False otherwise
    """
    #SHA-256 constants
    
    #compare the generated constants with the official SHA-256 constants
    AllMatch = True
    for i in range(64):
        if generated_constants[i] == hex(sha256_constants[i]):
            print(f"[{i}] matches")
        else:
            print(f"[{i}] mismatch")
            AllMatch = False
    return print("AllMatch" if AllMatch else "Not all matched")

SHScomparison(hex_values)
    



[0] matches
[1] matches
[2] matches
[3] matches
[4] matches
[5] matches
[6] matches
[7] matches
[8] matches
[9] matches
[10] matches
[11] matches
[12] matches
[13] matches
[14] matches
[15] matches
[16] matches
[17] matches
[18] matches
[19] matches
[20] matches
[21] matches
[22] matches
[23] matches
[24] matches
[25] matches
[26] matches
[27] matches
[28] matches
[29] matches
[30] matches
[31] matches
[32] matches
[33] matches
[34] matches
[35] matches
[36] matches
[37] matches
[38] matches
[39] matches
[40] matches
[41] matches
[42] matches
[43] matches
[44] matches
[45] matches
[46] matches
[47] matches
[48] matches
[49] matches
[50] matches
[51] matches
[52] matches
[53] matches
[54] matches
[55] matches
[56] matches
[57] matches
[58] matches
[59] matches
[60] matches
[61] matches
[62] matches
[63] matches
AllMatch


In [139]:
def test_problem2_constants():
    """
    Test suite for Problem 2: SHA-256 constant generation and verification.
    
    Tests both prime generation and constant derivation against official values.
    """

    print("Testing Problem 2: SHA-256 Constant Generation")
 
    
    # Test 1: Prime generation
    print("\n--- Test 1: Prime Number Generation ---")
    first_10_primes = primes(10)
    expected_primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
    
    if first_10_primes == expected_primes:
        print(f"PASS: First 10 primes correct: {first_10_primes}")
    else:
        print(f"FAIL: Expected {expected_primes}, got {first_10_primes}")
    
    # Test 2: Verify first 64 primes are correct
    primes_64 = primes(64)
    print(f"Generated 64 primes: {primes_64[0]} to {primes_64[-1]}")
    
    # Test 3: Generate and verify constants
    print("\n--- Test 2: SHA-256 Constant Verification ---")
    hex_values = generate_sha256_constants(64)
    
    # Count matches
    matches = sum(1 for i in range(64) if hex_values[i] == hex(sha256_constants[i]))
    
    if matches == 64:
        print(f"PASS: All 64 constants match official SHA-256 values")
    else:
        print(f"FAIL: Only {matches}/64 constants match")
        print("\nRunning detailed comparison...")
        SHScomparison(hex_values)  # Call your existing function for details
    
    print("Problem 2 Tests Complete")


# Run the test
test_problem2_constants()

Testing Problem 2: SHA-256 Constant Generation

--- Test 1: Prime Number Generation ---
PASS: First 10 primes correct: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
Generated 64 primes: 2 to 311

--- Test 2: SHA-256 Constant Verification ---
PASS: All 64 constants match official SHA-256 values
Problem 2 Tests Complete


## Problem 3: Padding

### Why Padding is Necessary

SHA-256 processes messages in fixed-size blocks of 512 bits (64 bytes). However, messages can be any length. Padding serves two critical purposes:

1. **Ensures complete blocks**: Messages must be padded to a multiple of 512 bits
2. **Prevents attacks**: Encodes the message length to prevent length extension attacks

### Length Extension Attacks

Without proper padding, an attacker could:
1. Take a hash H(M) of message M
2. Append data to M without knowing M
3. Calculate H(M || data) without knowing M

By including the original message length in the padding, SHA-256 prevents this attack because the attacker cannot correctly forge the length field.

### The Padding Algorithm (FIPS 180-4, Section 5.1.1)

For a message M of length ℓ bits:

1. **Append bit '1'**: Add a single 1 bit (as byte 0x80 = 10000000 in binary)
   - Marks the end of the original message
   
2. **Append '0' bits**: Add k zero bits where k is the smallest non-negative solution to:
   - ℓ + 1 + k ≡ 448 (mod 512)
   - This ensures the padded message is 448 bits mod 512
   
3. **Append length**: Add the original length ℓ as a 64-bit big-endian integer
   - Takes up the final 64 bits (8 bytes) of the last block

**Result**: Total length is a multiple of 512 bits

### Padding Examples

**Example 1: Message "abc" (24 bits)**
```
Original:     'a'  'b'  'c'
Hex:          61   62   63
Binary:       01100001 01100010 01100011

After padding:
61 62 63 80 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18
                                            └─ length = 24 bits
```

**Example 2: Message of 55 bytes**
- 55 bytes = 440 bits
- After adding 1 bit: 441 bits
- Need 7 more bits to reach 448 (mod 512)
- Then add 64-bit length
- Total: 512 bits (exactly 1 block)

**Example 3: Message of 56 bytes**
- 56 bytes = 448 bits
- After adding 1 bit: 449 bits
- Need 511 bits of padding to reach 960 bits (448 mod 512)
- Then add 64-bit length
- Total: 1024 bits (2 blocks)

### Big-Endian Encoding

The message length is encoded as big-endian (most significant byte first), which is the network byte order and ensures consistency across different computer architectures.

Example: Length 24 bits = 0x0000000000000018 in big-endian

### Implementation as a Generator

implemented block parsing as a Python generator function, which:
- Yields one block at a time (memory efficient)
- Follows Python best practices for processing large data


In [140]:
def block_parse(msg):
    """ Yields 512 bit (64 byte) blocks from the input message after padding according to SHA-256 specifications
    Parameters
    ----------
    msg: bytes
        input message to be padded and split into blocks            
    Yields
    -------
    bytes
        64 byte (512 bit) blocks of the padded message
    """
    #https://realpython.com/introduction-to-python-generators/
    
    #get msg length in bits and save before padding
    msg_len= len(msg) * 8
    
    #convert msg to bytearray to allow appending of bytes
    padded = bytearray(msg)
    
    #append the bit '1' to the end as the secure hashing standard specifies
    #to mark the end of the original message and start of padding
    padded.append(0x80)

    #get current length of the padded message
    current_len = len(padded)
    
    #check how far into current 64 byte(512 bit) block we are
    #check how many more bytes are needed to reach 56 bytes(448 bits)(usable number of bits in block after padding)
    #wrap to next block if already at or past 56 bytes
    padding_needed = (56 - current_len % 64) % 64
    #reserve 8 bytes(64 bits) for original message length
    padded.extend(b'\x00' * padding_needed)
    #append original message length as a 64 bit big-endian integer(nost significant byte first)
    padded.extend(msg_len.to_bytes(8, byteorder='big'))
    
    # Verify total length is multiple of 64 bytes (512 bits)
    assert len(padded) % 64 == 0, "padded message must be multiple of 512 bits"
    
    # Yield blocks of 64 bytes (512 bits)
    for i in range(0, len(padded), 64):
        yield bytes(padded[i:i+64])

### Testing Padding Implementation

The following tests verify that padding works correctly for various message lengths, including edge cases that require different amounts of padding or multiple blocks.

In [141]:
def test_problem3_padding():
    """
    Comprehensive test suite for message padding and block parsing.
    
    Tests various message lengths to ensure proper padding:
    - Empty message
    - Short messages (< 56 bytes)
    - Boundary case (exactly 55 bytes - requires 1 block)
    - Medium messages
    - Long messages (multiple blocks)
    """
    print("=" * 60)
    print("Testing Problem 3: Message Padding")
    print("=" * 60)
    
    test_cases = [
        (b"", "Empty message"),
        (b"abc", "3-byte message (from FIPS 180-4 examples)"),
        (b"a" * 55, "55-byte message (boundary case - fits in 1 block)"),
        (b"a" * 56, "56-byte message (requires 2 blocks)"),
        (b"The quick brown fox jumps over the lazy dog", "43-byte message"),
        (b"a" * 64, "Exactly one block (64 bytes)"),
        (b"a" * 100, "Multi-byte message (100 bytes)"),
        (b"a" * 128, "Exactly two blocks (128 bytes)"),
    ]
    
    for msg, description in test_cases:
        print(f"\n--- {description} ---")
        print(f"Original length: {len(msg)} bytes ({len(msg) * 8} bits)")
        
        blocks = list(block_parse(msg))
        total_padded_len = len(blocks) * 64
        
        print(f"Number of blocks: {len(blocks)}")
        print(f"Total padded length: {total_padded_len} bytes ({total_padded_len * 8} bits)")
        
        # Verify each block is 512 bits (64 bytes)
        all_blocks_valid = True
        for i, block in enumerate(blocks):
            if len(block) != 64:
                print(f"FAIL: Block {i} has length {len(block)} (expected 64)")
                all_blocks_valid = False
        
        if all_blocks_valid:
            print(f"PASS: All {len(blocks)} block(s) are 64 bytes")
        
        # Verify last 8 bytes contain message length
        last_block = blocks[-1]
        encoded_length = int.from_bytes(last_block[-8:], byteorder='big')
        expected_length = len(msg) * 8
        
        if encoded_length == expected_length:
            print(f"PASS: Length encoded correctly: {encoded_length} bits")
        else:
            print(f"FAIL: Length mismatch: encoded {encoded_length}, expected {expected_length}")
        
        # Verify padding bit (0x80) is present
        # For short messages (< 56 bytes), padding is in the first block
        # For longer messages, need to find where message ends
        padding_found = False
        padding_position = -1
        
        if len(msg) < 56:
            # Padding should be in first block right after message
            first_block = blocks[0]
            if len(msg) < len(first_block) and first_block[len(msg)] == 0x80:
                padding_found = True
                padding_position = len(msg)
        else:
            # For longer messages, padding is in the block where message ends
            msg_blocks_needed = len(msg) // 64
            remaining_bytes = len(msg) % 64
            
            if remaining_bytes < 56:
                # Padding in the same block as last message bytes
                target_block = blocks[msg_blocks_needed]
                if target_block[remaining_bytes] == 0x80:
                    padding_found = True
                    padding_position = remaining_bytes
            else:
                # Padding in next block at position 0
                if len(blocks) > msg_blocks_needed + 1:
                    target_block = blocks[msg_blocks_needed + 1]
                    if target_block[0] == 0x80:
                        padding_found = True
                        padding_position = 0
        
        if padding_found:
            print(f"PASS: Padding bit (0x80) found at correct position")
        else:
            print(f"FAIL: Padding bit (0x80) not found at expected position")
        
        # Verify zeros between padding bit and length
        # This is a spot check - verify some zeros exist
        zero_check_passed = True
        if len(blocks) == 1 and len(msg) < 56:
            # Check bytes between padding and length field
            first_block = blocks[0]
            for i in range(len(msg) + 1, 56):  # Skip the 0x80 byte
                if first_block[i] != 0x00:
                    zero_check_passed = False
                    break
        
        if zero_check_passed:
            print(f"PASS: Zero padding verified")
    

    
    # Additional test: Verify against known SHA-256 test vector
    print("\n--- Special Test: Known Test Vector ---")
    print("Testing 'abc' against known padding from FIPS 180-4")
    
    msg = b"abc"
    blocks = list(block_parse(msg))
    
    # Known padding for "abc" (from FIPS 180-4, page 20)
    # Should be: 61 62 63 80 00 00 ... 00 00 00 00 00 00 00 18
    expected_first_bytes = bytes([0x61, 0x62, 0x63, 0x80])
    expected_last_bytes = bytes([0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x18])
    
    if blocks[0][:4] == expected_first_bytes:
        print(f"PASS: First 4 bytes match expected: {blocks[0][:4].hex()}")
    else:
        print(f"FAIL: First 4 bytes: {blocks[0][:4].hex()}, expected: {expected_first_bytes.hex()}")
    
    if blocks[0][-8:] == expected_last_bytes:
        print(f"PASS: Last 8 bytes match expected: {blocks[0][-8:].hex()}")
    else:
        print(f"FAIL: Last 8 bytes: {blocks[0][-8:].hex()}, expected: {expected_last_bytes.hex()}")
    
    
    print("Problem 3 Tests Complete")

# Run the test
test_problem3_padding()

Testing Problem 3: Message Padding

--- Empty message ---
Original length: 0 bytes (0 bits)
Number of blocks: 1
Total padded length: 64 bytes (512 bits)
PASS: All 1 block(s) are 64 bytes
PASS: Length encoded correctly: 0 bits
PASS: Padding bit (0x80) found at correct position
PASS: Zero padding verified

--- 3-byte message (from FIPS 180-4 examples) ---
Original length: 3 bytes (24 bits)
Number of blocks: 1
Total padded length: 64 bytes (512 bits)
PASS: All 1 block(s) are 64 bytes
PASS: Length encoded correctly: 24 bits
PASS: Padding bit (0x80) found at correct position
PASS: Zero padding verified

--- 55-byte message (boundary case - fits in 1 block) ---
Original length: 55 bytes (440 bits)
Number of blocks: 1
Total padded length: 64 bytes (512 bits)
PASS: All 1 block(s) are 64 bytes
PASS: Length encoded correctly: 440 bits
PASS: Padding bit (0x80) found at correct position
PASS: Zero padding verified

--- 56-byte message (requires 2 blocks) ---
Original length: 56 bytes (448 bits)
Nu

## Problem 4: Hashes

### The SHA-256 Compression Function

This is the core of SHA-256, where the actual hashing happens. The compression function takes:
- **Current hash value**: Eight 32-bit words (H₀ through H₇)
- **Message block**: 512 bits (64 bytes)

And produces:
- **Updated hash value**: Eight 32-bit words

This function is called once for each 512-bit block of the padded message.

### Understanding the Algorithm (FIPS 180-4, Section 6.2.2)

#### Step 1: Prepare Message Schedule (W)

The 512-bit block is expanded into 64 32-bit words (W₀ through W₆₃):

- **First 16 words**: Directly from the message block
  - Each word is 4 consecutive bytes in big-endian order
  
- **Remaining 48 words**: Generated using the formula:
  ```
  Wₜ = σ₁(Wₜ₋₂) + Wₜ₋₇ + σ₀(Wₜ₋₁₅) + Wₜ₋₁₆
  ```
  - This creates diffusion, spreading each input bit's influence

**Why 64 words?** This expansion ensures that every bit of the input influences many bits of the output through multiple rounds of processing.

#### Step 2: Initialize Working Variables

Eight working variables (a, b, c, d, e, f, g, h) are initialized with the current hash values:
```
a = H₀, b = H₁, c = H₂, d = H₃
e = H₄, f = H₅, g = H₆, h = H₇
```

#### Step 3: Main Loop (64 Rounds)

For each of the 64 words in the message schedule (t = 0 to 63):

1. **Calculate T₁**:
   ```
   T₁ = h + Σ₁(e) + Ch(e,f,g) + Kₜ + Wₜ
   ```
   - h: Current value of working variable h
   - Σ₁(e): Non-linear transformation of e
   - Ch(e,f,g): Choose function - e selects bits from f or g
   - Kₜ: Round constant (from Problem 2)
   - Wₜ: Current word from message schedule

2. **Calculate T₂**:
   ```
   T₂ = Σ₀(a) + Maj(a,b,c)
   ```
   - Σ₀(a): Non-linear transformation of a
   - Maj(a,b,c): Majority function

3. **Update working variables** (rotate and mix):
   ```
   h = g
   g = f
   f = e
   e = d + T₁
   d = c
   c = b
   b = a
   a = T₁ + T₂
   ```

 Each round:
- Mixes the message data (Wₜ) with the state
- Applies non-linear functions (Σ, Ch, Maj)
- Rotates values through the working variables
- Adds round constants to prevent symmetry

#### Step 4: Compute Intermediate Hash

After 64 rounds, add the working variables back to the hash:
```
H₀ = H₀ + a
H₁ = H₁ + b
H₂ = H₂ + c
H₃ = H₃ + d
H₄ = H₄ + e
H₅ = H₅ + f
H₆ = H₆ + g
H₇ = H₇ + h
```

This addition (mod 2³²) ensures that each block influences all subsequent blocks.

### Security Properties

The compression function provides:

1. **Avalanche Effect**: Changing one bit in the input changes ~50% of output bits
2. **Collision Resistance**: Finding two inputs with the same output is computationally infeasible
3. **Pre-image Resistance**: Cannot work backwards from hash to input
4. **Second Pre-image Resistance**: Cannot find different input with same hash


In [ ]:
def hash(current, block):
    """
    Calculates the next hash value given the current hash value and the next message block
    according to  secure hash standard 6.2.2 sha-256 .
    
    Parameters
    ----------
        current: tuple of 8 32-bit integers  - current hash values
        block: message block of exactly 64 bytes (512 bits) - (output from block_parse)
    
    Returns
    -------
        tuple of 8 32-bit integers  - updated hash values
    

    """
    #get constants
    K_hex = generate_sha256_constants(64)   # returns hex strings
    K = [int(h, 16) for h in K_hex]         # convert hex → int

    #step 1
    #the message schedule W (64 32-bit words) shs page 22
    #made from the first 16 words of the block and extended to 64 words
    W = []
    
    #First 16 words from the block (convert 4 bytes to 32-bit integer)
    for i in range(16):
        #append each 32 bit word to W in order of most significant byte first
        #append bytes starting from index i*4 to (i+1)*4 i.e block[0:4], block[4:8], ... 
        W.append(int.from_bytes(block[i*4:(i+1)*4], byteorder='big'))
    
    #Extend to 64 words using the schedule formula on page 22
    for i in range(16, 64):
        #get the next word by getting the word 2 postions before, 7 positions before, 15 positions before and 16 positions before
        #apply the sigma functions to the 2 and 15 position before words and sum all masking to keep only the lower 32 bits
        W.append((Sigma1_2(W[i-2]) + W[i-7] + Sigma0_2(W[i-15]) + W[i-16]) & 0xFFFFFFFF)
    
    #Step 2
    #Initialize the eight working variables with the (i-1)th hash value
    a, b, c, d, e, f, g, h = current
    
    #Step 3 page 23
    for t in range(64):
        #Calculate the temporary values T1 and T2
        T1 = (h + Sigma1(e) + Ch(e, f, g) + K[t] + W[t]) & 0xFFFFFFFF
        T2 = (Sigma0(a) + Maj(a, b, c)) & 0xFFFFFFFF
        #Update the working variables
        h = g
        g = f
        f = e
        e = (d + T1) & 0xFFFFFFFF
        d = c
        c = b
        b = a
        a = (T1 + T2) & 0xFFFFFFFF
    
    #Step 4 page 23
    #Compute the i-th intermediate hash value H(i) by adding working variables to current hash
    H0 = (current[0] + a) & 0xFFFFFFFF
    H1 = (current[1] + b) & 0xFFFFFFFF
    H2 = (current[2] + c) & 0xFFFFFFFF
    H3 = (current[3] + d) & 0xFFFFFFFF
    H4 = (current[4] + e) & 0xFFFFFFFF
    H5 = (current[5] + f) & 0xFFFFFFFF
    H6 = (current[6] + g) & 0xFFFFFFFF
    H7 = (current[7] + h) & 0xFFFFFFFF
    
    return (H0, H1, H2, H3, H4, H5, H6, H7)

### Complete SHA-256 Implementation

 all the pieces (Problems 1-4)combined into a complete SHA-256 function that can hash any message.

In [143]:
def sha256(msg):
    """
    Complete SHA-256 implementation using functions from Problems 1-4.
    
    Parameters
    ----------
    msg : bytes or str
        Message to hash (will be converted to UTF-8 bytes if string)
        
    Returns
    -------
    str
        64-character hexadecimal hash digest
    """
    # Convert string to bytes if needed (UTF-8 encoding as specified)
    if isinstance(msg, str):
        msg = msg.encode('utf-8')
    
    # Start with initial hash values from Problem 4
    current = tuple(sha256_initial_hash_values)
    
    # Process each 512-bit block from Problem 3
    for block in block_parse(msg):
        # Update hash using Problem 4 function
        current = hash(current, block)
    
    # Convert final hash tuple to hexadecimal string
    return ''.join(f'{h:08x}' for h in current)

## Problem 5: Passwords

### The Problem with Simple Hashing

The three password hashes provided were created by:
1. Taking a password as a UTF-8 string
2. Computing SHA-256 hash (one pass)
3. Storing the hash

This approach has a critical weakness: **it's vulnerable to dictionary attacks**.

### Dictionary Attack Method

A dictionary attack works by:
1. Creating a list of common passwords
2. Hashing each password
3. Comparing hashes to find matches

**Why it works:**
- People choose predictable passwords ("password", "123456", etc.)
- SHA-256 is fast (millions of hashes per second)
- No salt means the same password always produces the same hash

### Finding the Passwords

I used a dictionary attack with a list of common passwords:

In [ ]:
def crack_passwords():
    """
    Crack the three given SHA-256 password hashes using dictionary attack.
    
    The hashes are from common passwords encoded in UTF-8 and hashed with
    a single pass of SHA-256.
    
    Returns
    -------
    dict
        Dictionary mapping password number to cracked password
    """
    #Target hashes 
    target_hashes = {
        1: '5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8',
        2: '873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34',
        3: 'b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342'
    }
    
    
    common_passwords = [
    'password', '123456', '12345678', 'qwerty', 'abc123',
    'monkey', '1234567', 'letmein', 'trustno1', 'dragon',
    'baseball', 'iloveyou', 'master', 'sunshine', 'ashley',
    'bailey', 'shadow', '123123', '654321', 'superman',
    'princess', 'michael', 'football', 'welcome', 'jesus',
    'ninja', 'mustang', 'password1', 'cheese', 'computer',
    'admin', 'root', 'test', 'guest', 'hello',
    'login', 'passw0rd', 'pass', '1234', '12345',
    'summer', 'winter', 'spring', 'fall', 'love',
    'secret', 'blue', 'green', 'red', 'black',
    '111111', '121212', '000000', '666666', '123321',
    'qwertyuiop', 'asdfghjkl', '1q2w3e4r', '1qaz2wsx', 'qazwsx',
    'zaq12wsx', 'iloveyou1', 'welcome1', 'password123', 'qwerty123',
    'charlie', 'donald', 'pokemon', 'jordan23', 'harley',
    'letmein1', 'football1', 'monkey1', 'loveme', 'flower',
    'pepper', 'cookie', 'buster', 'hottie', 'ginger',
    'jessica', 'pepper1', 'hunter', 'soccer', 'killer',
    'batman', 'starwars', 'merlin', 'trustno1!'
    '123qwe', 'football123', 'baseball1', 'welcome123', 'admin123',
    'qwerty1', 'pass123', '1qazxsw2', 'letmein123', 'abc12345',
    '123abc', 'password11', 'password12', 'qwerty12', 'qwerty1234',
    'asdf1234', 'asdf123', 'zxcvbnm', 'zxc123', 'asdfgh',
    '1q2w3e', '1qaz2wsx3edc', 'qwe123', 'qwe12345', 'password1234',
    '111222', '123654', '987654', '147258', '258456',
    '159753', '741852', '852456', '963852', '159357',
    '147369', '258147', '369258', '123789', '789123' ,'P@ssw0rd'
]
    
    results = {}
    

    for num, target in target_hashes.items():
        print(f"\nCracking password {num}: {target}")
        found = False
        
        #Try each common password
        for pwd in common_passwords:
            #Compute SHA-256 hash of the password
            computed_hash = sha256(pwd.encode('utf-8'))
            #Compare with target hash
            if computed_hash == target:
                #If match found, store result and break
                results[num] = pwd
                print(f"Password {num} is: '{pwd}'")
                found = True
                break
        
        if not found:
            print(f"Password {num} not found in common password list")
    
    print(f"\nFound {len(results)}/3 passwords")
    #return found passwords
    return results

### Results and Analysis

The three passwords were found to be common, weak passwords that appear in password breach databases. This demonstrates why simple hashing is insufficient for password storage.

### Improving Password Security

To properly protect passwords, we need multiple layers of defense:

#### 1. **Salting**

Add a random value (salt) to each password before hashing:
```
hash = SHA-256(password + salt)
```

**Benefits:**
- Same password gets different hash for each user
- Prevents rainbow table attacks
- Must crack each password individually

**Example:**
```python
import os
salt = os.urandom(16)  # 16 random bytes
hash = sha256(password.encode() + salt)
# Store: salt + hash
```

#### 2. **Key Stretching (Multiple Iterations)**

Hash the password thousands of times:
```
hash = password
for i in range(100000):
    hash = SHA-256(hash)
```

**Benefits:**
- Slows down attackers (each password takes longer to test)
- Doesn't significantly impact legitimate logins
- Makes brute force attacks impractical


### Conclusion

While SHA-256 is excellent for data integrity and digital signatures, it's not designed for password storage. Its speed (a benefit for most uses) becomes a weakness when attackers can test billions of passwords per second.


"""

In [ ]:
print(crack_passwords())


Cracking password 1: 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
Password 1 is: 'password'

Cracking password 2: 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
Password 2 is: 'cheese'

Cracking password 3: b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342
Password 3 is: 'P@ssw0rd'

Found 3/3 passwords
{1: 'password', 2: 'cheese', 3: 'P@ssw0rd'}


In [ ]:
#TODO 
# Problem 1
# Referencing - secure hashing standard , 
# Problem 2
# referemce - secure hashing standard,geeks for geeks
#problem 3
# references – secure hashing standard,realpython.com,geeks for geeks byte order,python doc
# problem 4
# references – secure hashing standard

#Readme

